In [6]:
#Import packages
import os
import sys
import logging
import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import cartopy.feature as cfeature

from scipy.sparse import csgraph
from itertools import product
import re

from shapely.geometry import Point, LineString
import shapely, shapely.prepared
from shapely.wkt import loads
from shapely.validation import make_valid

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)
#for solved network plot

from matplotlib.patches import Circle, Ellipse
from matplotlib.legend_handler import HandlerPatch
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
plt.style.use("bmh")

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid", font_scale=1)

In [2]:
colors = { "tech_colors": 
            {
                "SMR CC": "darkblue",
                "gas for industry CC": "brown",
                "process emissions CC": "gray",
                "CO2 pipeline": "gray",
                "onwind": "dodgerblue",
                "onwind2": "dodgerblue",
                "onshore wind": "#235ebc",
                "offwind": "#6895dd",
                "offwind2": "#6895dd",
                "csp": "coral",
                "offshore wind": "#6895dd",
                "offwind-ac": "c",
                "offshore wind (AC)": "#6895dd",
                "offwind-dc": "#74c6f2",
                "offshore wind (DC)": "#74c6f2",
                "wave": '#004444',
                "hydro": '#3B5323',
                "hydro reservoir": '#3B5323',
                "ror": '#78AB46',
                "run of river": '#78AB46',
                "hydroelectricity": 'blue',
                "solar": "orange",
                "solar PV": "#f9d002",
                "solar thermal": "coral",
                "solar rooftop": '#ffef60',
                "OCGT": "wheat",
                "OCGT marginal": "sandybrown",
                "OCGT-heat": '#ee8340',
                "gas boiler": '#ee8340',
                "gas boilers": '#ee8340',
                "gas boiler marginal": '#ee8340',
                "gas-to-power/heat": 'brown',
                "gas": "brown",
                "natural gas": "brown",
                "SMR": '#4F4F2F',
                "oil": '#B5A642',
                "oil boiler": '#B5A677',
                "lines": "k",
                "transmission lines": "#614700",
                "H2": "m",
                "H2 liquefaction": "m",
                "hydrogen storage": "m",
                "battery": "slategray",
                "battery storage": "slategray",
                "home battery": '#614700',
                "home battery storage": '#614700',
                "Nuclear": "r",
                "Nuclear marginal": "r",
                "nuclear": "r",
                "uranium": "r",
                "Coal": "k",
                "coal": "k",
                "Coal marginal": "k",
                "Lignite": "grey",
                "lignite": "grey",
                "Lignite marginal": "grey",
                "CCGT": '#ee8340',
                "CCGT marginal": '#ee8340',
                "heat pumps": '#76EE00',
                "heat pump": '#76EE00',
                "air heat pump": '#76EE00',
                "ground heat pump": '#40AA00',
                "power-to-heat": 'red',
                "resistive heater": "pink",
                "Sabatier": '#FF1493',
                "methanation": '#FF1493',
                "power-to-gas": 'purple',
                "power-to-liquid": 'darkgreen',
                "helmeth": '#7D0552',
                "DAC": 'deeppink',
                "co2 stored": '#123456',
                "CO2 sequestration": '#123456',
                "CC": "k",
                "co2": '#123456',
                "co2 vent": '#654321',
                "agriculture heat": '#D07A7A',
                "agriculture machinery oil": '#1e1e1e',
                "agriculture machinery oil emissions": '#111111',
                "agriculture electricity": '#222222',
                "solid biomass for industry co2 from atmosphere": '#654321',
                "solid biomass for industry co2 to stored": '#654321',
                "solid biomass for industry CC": '#654321',
                "gas for industry co2 to atmosphere": '#654321',
                "gas for industry co2 to stored": '#654321',
                "Fischer-Tropsch": '#44DD33',
                "kerosene for aviation": '#44BB11',
                "naphtha for industry": '#44FF55',
                "land transport oil": '#44DD33',
                "water tanks": '#BBBBBB',
                "hot water storage": '#BBBBBB',
                "hot water charging": '#BBBBBB',
                "hot water discharging": '#999999',
                "CO2 pipeline": '#999999',
                "CHP": "r",
                "CHP heat": "r",
                "CHP electric": "r",
                "PHS": "g",
                "Ambient": "k",
                "Electric load": "b",
                "Heat load": "r",
                "heat": "darkred",
                "rural heat": '#880000',
                "central heat": '#b22222',
                "decentral heat": '#800000',
                "low-temperature heat for industry": '#991111',
                "process heat": '#FF3333',
                "heat demand": "darkred",
                "electric demand": "k",
                "Li ion": "grey",
                "district heating": '#CC4E5C',
                "retrofitting": "purple",
                "building retrofitting": "purple",
                "BEV charger": "grey",
                "V2G": "grey",
                "land transport EV": "grey",
                "electricity":" k",
                "gas for industry": '#333333',
                "solid biomass for industry": '#555555',
                "industry electricity": '#222222',
                "industry new electricity": '#222222',
                "process emissions to stored": '#444444',
                "process emissions to atmosphere": '#888888',
                "process emissions": '#222222',
                "oil emissions": '#666666',
                "industry oil emissions": '#666666',
                "land transport oil emissions": '#666666',
                "land transport fuel cell": '#AAAAAA',
                "biogas": '#800000',
                "solid biomass": '#DAA520',
                "biomass": '#DAA520',
                "today": '#D2691E',
                "shipping": '#6495ED',
                "shipping oil": "#6495ED",
                "shipping oil emissions": "#6495ED",
                "electricity distribution grid": 'y',
                "solid biomass transport": "green",
                "H2 for industry": "#222222",
                "H2 for shipping": "#6495ED",
                "biomass EOP": "green",
                "agriculture oil": "darkgray",
                "residential rural heat": "darkgray",
                "residential rural water tanks": "darkgray",
                "residential rural solar thermal": "darkgray",
                "services rural heat": "darkgray",
                "services rural water tanks":"darkgray",
                "services rural solar thermal":"darkgray",
                "residential urban decentral heat":"darkgray",
                "residential urban decentral water tanks":"darkgray",
                "residential urban decentral solar thermal":"darkgray",
                "services urban decentral heat":"darkgray",
                "services urban decentral water tanks":"darkgray",
                "services urban decentral solar thermal":"darkgray",
                "urban central heat":"darkgray",
                "urban central water tanks":"darkgray",
                "urban central solar thermal":"darkgray",
                "geothermal":"black"
            }
        }

In [30]:
def plot_network(n, line=False, fn=None):
    bus_scale = 9e1 
    line_scale = 1e3
    regions_onshore = gpd.read_file("/home/raj08555/dev/sandbox-cs/pypsa-earth-sec/pypsa-earth/resources/shapes/gadm_shapes.geojson")
    regions_offshore = gpd.read_file("/home/raj08555/dev/sandbox-cs/pypsa-earth-sec/pypsa-earth/resources/shapes/offshore_shapes.geojson")
    fig, ax = plt.subplots(figsize=(17, 9), subplot_kw={"projection": ccrs.PlateCarree()})
    gen = n.generators.groupby(["bus", "carrier"]).p_nom.sum()
    sto = n.storage_units.groupby(["bus", "carrier"]).p_nom.sum()
    buses = pd.concat([gen, sto])
    lines=0; links=0;
    if line:
        lines = n.lines.s_nom / line_scale
        links=n.links.p_nom / line_scale,

    with plt.rc_context({"patch.linewidth": 0.}):
        n.plot(
            bus_sizes=buses / bus_scale,
            bus_alpha=0.9,
            line_widths=lines,#n.lines.s_nom / line_scale,
            link_widths=links,#n.links.p_nom / line_scale,
            line_colors="teal",
            bus_colors = n.carriers.color,
            ax=ax,
            margin=0.2,
            color_geomap=True,
        )
    regions_onshore.plot(
        ax=ax,
        facecolor="white",
        edgecolor="black",
        aspect="equal",
        transform=ccrs.PlateCarree(),
        linewidth=0,
    )
    regions_offshore.plot(
        ax=ax,
        facecolor="blue",
        edgecolor="blue",
        aspect="equal",
        transform=ccrs.PlateCarree(),
        linewidth=1,
    )
    #ax.set_extent(regions_onshore.total_bounds[[0, 2, 1, 3]])
    ax.set_extent(regions_onshore.total_bounds[[0, 2, 1, 3]])
    legend_kwargs = {"loc": "upper left", "frameon": False}
    # circles legend may requite some fine-tuning
    legend_circles_dict = {"bbox_to_anchor": (1, 1.1), "labelspacing":11, "handletextpad":7, "title":"Capacity", **legend_kwargs}
    #bus_sizes = [5000, 10e3]  # in MW
    bus_sizes = [10, 100, 500]  # in MW
    line_sizes = [1000, 3000]  # in MW
    add_legend_circles(
        ax,
        [s / bus_scale for s in bus_sizes],
        [f"{s} MW" for s in bus_sizes],
        legend_kw=legend_circles_dict,    
    )
    if line:
        add_legend_lines(
            ax,
            [s / line_scale for s in line_sizes],
            [f"{s / 1000} GW" for s in line_sizes],
            legend_kw={"bbox_to_anchor": (1, 1.0), "handletextpad":7, **legend_kwargs},
        )
    add_legend_patches(
        ax,
        n.carriers.color,
        n.carriers.nice_name,
        legend_kw={"bbox_to_anchor": (1, 0), **legend_kwargs, "loc":"lower left", "handletextpad":1.5, "title":"Technology",},
    )
    fig.tight_layout()
#fig.savefig("brownfield_capacities_" + country_code + ".png", bbox_inches="tight", dpi=300)

In [28]:
colors["tech_colors"]

{'SMR CC': 'darkblue',
 'gas for industry CC': 'brown',
 'process emissions CC': 'gray',
 'CO2 pipeline': '#999999',
 'onwind': 'dodgerblue',
 'onwind2': 'dodgerblue',
 'onshore wind': '#235ebc',
 'offwind': '#6895dd',
 'offwind2': '#6895dd',
 'csp': 'coral',
 'offshore wind': '#6895dd',
 'offwind-ac': 'c',
 'offshore wind (AC)': '#6895dd',
 'offwind-dc': '#74c6f2',
 'offshore wind (DC)': '#74c6f2',
 'wave': '#004444',
 'hydro': '#3B5323',
 'hydro reservoir': '#3B5323',
 'ror': '#78AB46',
 'run of river': '#78AB46',
 'hydroelectricity': 'blue',
 'solar': 'orange',
 'solar PV': '#f9d002',
 'solar thermal': 'coral',
 'solar rooftop': '#ffef60',
 'OCGT': 'wheat',
 'OCGT marginal': 'sandybrown',
 'OCGT-heat': '#ee8340',
 'gas boiler': '#ee8340',
 'gas boilers': '#ee8340',
 'gas boiler marginal': '#ee8340',
 'gas-to-power/heat': 'brown',
 'gas': 'brown',
 'natural gas': 'brown',
 'SMR': '#4F4F2F',
 'oil': '#B5A642',
 'oil boiler': '#B5A677',
 'lines': 'k',
 'transmission lines': 'k',
 'H2':

In [1]:

def plot_elec_network(n, line=False, fn=None):
    regions_onshore = gpd.read_file("../../sandbox-cs/pypsa-earth-sec/pypsa-earth/resources/shapes/gadm_shapes.geojson")
    regions_offshore = gpd.read_file("../../sandbox-cs/pypsa-earth-sec/pypsa-earth/resources/shapes/offshore_shapes.geojson")
    substations_OSMclean_path = "../../sandbox-cs/pypsa-earth-sec/pypsa-earth/resources/osm/clean/all_clean_substations.geojson"

    lines_OSMclean_path = "../../sandbox-cs/pypsa-earth-sec/pypsa-earth/resources/osm/clean/all_clean_lines.geojson"
    bus_scale = 9e1 
    line_scale = 1e3
    fig, ax = plt.subplots(figsize=(20, 9), subplot_kw={"projection": ccrs.PlateCarree()})
    gen = n.generators.groupby(["bus", "carrier"]).p_nom.sum()
    sto = n.storage_units.groupby(["bus", "carrier"]).p_nom.sum()
    buses = pd.concat([gen, sto])
    lines=0; links=0;
    if line:
        lines = n.lines.s_nom / line_scale
        links=n.links.p_nom / line_scale,

    with plt.rc_context({"patch.linewidth": 0.}):
        n.plot(
            bus_sizes=buses / bus_scale,
            bus_alpha=0.9,
            line_widths=lines,#n.lines.s_nom / line_scale,
            link_widths=links,#n.links.p_nom / line_scale,
            line_colors="teal",
            ax=ax,
            margin=0.2,
            color_geomap=True,
        )
    regions_onshore.plot(
        ax=ax,
        facecolor="white",
        edgecolor="black",
        aspect="equal",
        transform=ccrs.PlateCarree(),
        linewidth=0,
    )
    # regions_offshore.plot(
    #     ax=ax,
    #     facecolor="blue",
    #     edgecolor="blue",
    #     aspect="equal",
    #     transform=ccrs.PlateCarree(),
    #     linewidth=1,
    # )
    #ax.set_extent(regions_onshore.total_bounds[[0, 2, 1, 3]])
    ax.add_geometries(regions_onshore['geometry'], facecolor='white', edgecolor='gray', crs=ccrs.PlateCarree())
    ax.set_extent(regions_onshore.total_bounds[[0, 2, 1, 3]])
    legend_kwargs = {"loc": "upper left", "frameon": False}
    # circles legend may requite some fine-tuning
    legend_circles_dict = {"bbox_to_anchor": (1, 1.1), "labelspacing":11, "handletextpad":7, "title":"Capacity", **legend_kwargs}
    #bus_sizes = [5000, 10e3]  # in MW
    bus_sizes = [10, 100, 500]  # in MW
    line_sizes = [1000, 3000]  # in MW
    add_legend_circles(
        ax,
        [s / bus_scale for s in bus_sizes],
        [f"{s} MW" for s in bus_sizes],
        legend_kw=legend_circles_dict,    
    )
    if line:
        add_legend_lines(
            ax,
            [s / line_scale for s in line_sizes],
            [f"{s / 1000} GW" for s in line_sizes],
            legend_kw={"bbox_to_anchor": (1, 1.0), "handletextpad":7, **legend_kwargs},
        )
    add_legend_patches(
        ax,
        n.carriers.color,
        n.carriers.nice_name,
        legend_kw={"bbox_to_anchor": (1, 0), **legend_kwargs, "loc":"lower left", "handletextpad":1.5, "title":"Technology",},
    )
    fig.tight_layout()
#fig.savefig("brownfield_capacities_" + country_code + ".png", bbox_inches="tight", dpi=300)


def calculatelength(df, reference):
    length = pd.DataFrame(columns=["66kV","132kV", "220kV", "330kV", "350kV", "400kV","scope",  "reference"])
    length["66kV"] = df.loc[(df.index > 0) & (df.index <= 66000)].sum()
    length["132kV"] = df.loc[(df.index > 66000) & (df.index <= 132000)].sum()
    length["220kV"] = df.loc[(df.index > 132000) & (df.index <= 220000)].sum()
    length["330kV"] = df.loc[(df.index > 220000) & (df.index <= 330000)].sum()
    length["350kV"] = df.loc[(df.index > 330000) & (df.index <= 350000)].sum()
    length["400kV"] = df.loc[(df.index > 350000)].sum()
    length["scope"] = "Namibia"
    length["reference"] = reference

    return length

def labelformap(ax, df):
    for (id, row) in df.iterrows():
        ax.text(row.centroids.x, row.centroids.y, row.circuits, color=row.color)

In [1]:
def calc_all_gens(n):

    all_gens = ['biomass', 'coal', 'oil', 'OCGT', 'ror', 'nuclear', 'csp',

    'rooftop-solar', 'solar', 'onwind', 'onwind2', 'offwind',

    'offwind2']

  #res_gens = list(n.carriers[n.carriers.co2_emissions == 0].index)

    res_gens_dict = {} 

    for gen_index in all_gens:

        res_index = n.generators[n.generators.carrier == gen_index].index

    # vars_conv_gens = n.generators_t.p[conv_index]
        n.generators.loc[n.generators.carrier == gen_index, "emissions"] = 0

        weightings = pd.DataFrame(

        np.outer(n.snapshot_weightings["generators"], [1.0] * len(res_index)),

        index=n.snapshots,

        columns=res_index,

        )
        res_gens_dict[gen_index] = round((weightings * n.generators_t.p[res_index]).sum().sum()/1e6)

    res_gens_dict["all"] = sum(res_gens_dict.values())
    return res_gens_dict

In [10]:
def gen_emissions(n):
    conv_gens = list(n.carriers[n.carriers.co2_emissions > 0].index)
    conv_index = n.generators[n.generators.carrier.isin(conv_gens)].index
    # vars_conv_gens = n.generators_t.p[conv_index]

    n.generators.loc[n.generators.carrier.isin(conv_gens), "emissions"] = 0
    n.generators.loc[conv_index, "emissions"] = n.generators.loc[
        conv_index, "carrier"
    ].apply(lambda x: n.carriers.loc[x].co2_emissions)
    n.generators.emissions = n.generators.emissions.fillna(0)

    weightings = pd.DataFrame(
        np.outer(n.snapshot_weightings["generators"], [1.0] * len(conv_index)),
        index=n.snapshots,
        columns=conv_index,
    )

    emission_factors = pd.DataFrame(
        np.outer(
            [1.0] * len(n.snapshot_weightings["generators"]),
            n.generators.loc[conv_index, "emissions"],
        ),
        index=n.snapshots,
        columns=conv_index,
    )

    x=emission_factors*weightings
    x
    return (n.generators_t.p[conv_index] *x).sum().sum()/1e6